In [21]:
import random
from enum import Enum

In [22]:
class Suit(Enum):
    HEARTS = "Hearts"
    DIAMONDS = "Diamonds"
    CLUBS = "Clubs"
    SPADES = "Spades"

class Card:
    def __init__(self, suit: Suit, rank: str):
        self.suit = suit
        self.rank = rank

    def __repr__(self):
        return f"Card({self.suit.value}, {self.rank})"

    def value(self):
        """Returns the Blackjack value of the card"""
        if self.rank in ["Jack", "Queen", "King"]:
            return 10
        elif self.rank == "Ace":
            return 11  # Ace can be 1 or 11, but we handle it dynamically in hand evaluation
        else:
            return int(self.rank)


In [23]:
class Deck:
    def __init__(self):
        ranks = ["Ace", "2", "3", "4", "5", "6", "7", "8", "9", "10", "Jack", "Queen", "King"]
        self.cards = [Card(suit, rank) for suit in Suit for rank in ranks]
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        return self.cards.pop() if self.cards else None  # Returns None if the deck is empty

# Test deck creation
deck = Deck()
print(deck.cards[:5])  # Print first five cards to verify shuffling


[Card(Spades, King), Card(Clubs, 9), Card(Diamonds, 9), Card(Diamonds, 7), Card(Diamonds, 4)]


In [24]:
class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def total_value(self):
        """Calculates the total hand value, treating Aces as 1 or 11 dynamically"""
        total = sum(card.value() for card in self.cards)
        ace_count = sum(1 for card in self.cards if card.rank == "Ace")

        while total > 21 and ace_count:
            total -= 10  # Convert an Ace from 11 to 1
            ace_count -= 1

        return total

    def __repr__(self):
        return f"Hand: {self.cards} (Total: {self.total_value()})"

# Test hand
hand = Hand()
hand.add_card(Card(Suit.HEARTS, "Ace"))
hand.add_card(Card(Suit.SPADES, "King"))
print(hand)  # Expected output: Ace and King (Total: 21)


Hand: [Card(Hearts, Ace), Card(Spades, King)] (Total: 21)


In [25]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()

    def hit(self, deck):
        card = deck.draw_card()
        if card:
            self.hand.add_card(card)
        return card

    def is_bust(self):
        return self.hand.total_value() > 21

    def __repr__(self):
        return f"{self.name}: {self.hand}"

class Dealer(Player):
    def __init__(self):
        super().__init__("Dealer")

    def should_hit(self):
        return self.hand.total_value() < 17  # Dealer's rule: hit on 16 or less


In [26]:
class BlackjackGame:
    def __init__(self, player_names):
        self.deck = Deck()
        self.players = [Player(name) for name in player_names]
        self.dealer = Dealer()
        self.initialize_game()

    def initialize_game(self):
        """Deals two initial cards to each player and the dealer."""
        for _ in range(2):
            for player in self.players:
                player.hit(self.deck)
            self.dealer.hit(self.deck)

    def player_turn(self, player):
        """Handles player actions: hit or stand."""
        while True:
            print(f"\n{player}")
            action = input(f"{player.name}, do you want to 'hit' or 'stand'? ").strip().lower()
            if action == "hit":
                player.hit(self.deck)
                if player.is_bust():
                    print(f"{player.name} busts!")
                    break
            elif action == "stand":
                break

    def dealer_turn(self):
        """Dealer automatically plays based on rules."""
        while self.dealer.should_hit():
            self.dealer.hit(self.deck)

    def determine_winners(self):
        """Determines the winners based on game rules."""
        dealer_total = self.dealer.hand.total_value()
        print(f"\nFinal Dealer Hand: {self.dealer}")

        for player in self.players:
            player_total = player.hand.total_value()
            if player.is_bust():
                print(f"{player.name} loses (busted).")
            elif dealer_total > 21 or player_total > dealer_total:
                print(f"{player.name} wins!")
            elif player_total == dealer_total:
                print(f"{player.name} ties with the dealer.")
            else:
                print(f"{player.name} loses.")

    def play(self):
        """Runs the full game loop."""
        for player in self.players:
            self.player_turn(player)

        self.dealer_turn()
        self.determine_winners()

# Test game with two players
game = BlackjackGame(["Alice", "Bob"])
game.play()



Alice: Hand: [Card(Diamonds, King), Card(Diamonds, 6)] (Total: 16)


Alice, do you want to 'hit' or 'stand'?  hit


Alice busts!

Bob: Hand: [Card(Spades, 10), Card(Clubs, 6)] (Total: 16)


Bob, do you want to 'hit' or 'stand'?  stand



Final Dealer Hand: Dealer: Hand: [Card(Diamonds, 8), Card(Spades, 4), Card(Hearts, 5)] (Total: 17)
Alice loses (busted).
Bob loses.


In [27]:
def start_multiplayer_blackjack():
    player_count = int(input("Enter the number of players: "))
    player_names = [input(f"Enter name for player {i+1}: ") for i in range(player_count)]

    game = BlackjackGame(player_names)
    game.play()

# Start a multiplayer game
start_multiplayer_blackjack()


Enter the number of players:  3
Enter name for player 1:  A
Enter name for player 2:  B
Enter name for player 3:  C



A: Hand: [Card(Spades, 2), Card(Clubs, Ace)] (Total: 13)


A, do you want to 'hit' or 'stand'?  hit



A: Hand: [Card(Spades, 2), Card(Clubs, Ace), Card(Clubs, Jack)] (Total: 13)


A, do you want to 'hit' or 'stand'?  hit


...


B: Hand: [Card(Spades, 3), Card(Clubs, 6), Card(Hearts, King)] (Total: 19)


B, do you want to 'hit' or 'stand'?  hit


B busts!

C: Hand: [Card(Clubs, 8), Card(Diamonds, 4)] (Total: 12)


C, do you want to 'hit' or 'stand'?  hit


C busts!

Final Dealer Hand: Dealer: Hand: [Card(Diamonds, Ace), Card(Hearts, Jack)] (Total: 21)
A loses (busted).
B loses (busted).
C loses (busted).
